In [14]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [15]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = OPENAI_API_KEY)

In [16]:
def answer(msg):
    response = client.chat.completions.create(
        
        model = "gpt-4o-mini",
        temperature = 0,
        messages=[
            {
                "role": "user",
                "content": msg
            }
        ]
    )
    return response.choices[0].message.content

In [17]:
answer("What is the capital of France?")

'The capital of France is Paris.'